In [1]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.1 MB/s eta 0:00:00


In [59]:
import geopandas as gpd
from pyproj import Geod
from shapely import wkt

In [3]:
df_school = gpd.read_file('/content/school_mos_raw.geojson')
df_school

,DatasetId,VersionNumber,ReleaseNumber,RowId,Attributes,geometry
0,2263,4,15,NaN,"{'is_deleted': 0, 'ShortName': 'ГБОУ ДО ДТДМ «...","POLYGON ((37.47940 55.78360, 37.47888 55.78369..."
1,2263,4,15,NaN,"{'is_deleted': 0, 'ShortName': 'ГБУ «Лаборатор...","POLYGON ((37.67580 55.73956, 37.67559 55.73959..."
2,2263,4,15,NaN,"{'is_deleted': 0, 'ShortName': 'ГБПОУ колледж ...","MULTIPOLYGON (((37.71597 55.62112, 37.71596 55..."
3,2263,4,15,NaN,"{'is_deleted': 0, 'ShortName': 'ГБОУДО «ДТДиМ ...","POLYGON ((37.73285 55.80243, 37.73273 55.80245..."
4,2263,4,15,NaN,"{'is_deleted': 0, 'ShortName': 'ГБПОУ ФК № 35'...","POLYGON ((37.66745 55.60456, 37.66735 55.60467..."
...,...,...,...,...,...,...
578,2263,4,15,NaN,"{'is_deleted': 0, 'ShortName': 'ГБОУ школа № 1...","MULTIPOLYGON (((37.61439 55.65442, 37.61458 55..."
579,2263,4,15,NaN,"{'is_deleted': 0, 'ShortName': 'ГБОУ школа № 2...","MULTIPOLYGON (((37.65831 55.74988, 37.65855 55..."
580,2263,4,15,NaN,"{'is_deleted': 0, 'ShortName': 'ГКОУ СКОШИ № 7...","POLYGON ((37.63253 55.71649, 37.63187 55.71658..."
581,2263,4,15,NaN,"{'is_deleted': 0, 'ShortName': 'ГБОУ школа № 1...","MULTIPOLYGON (((37.74025 55.78712, 37.74022 55..."


In [ ]:
c = 0
m = 0
for i in range(df_school.shape[0]):
  try:
    mycoordslist = [list(x.exterior.coords) for x in df_school.iloc[i]['geometry'].geoms]
    if len(df_school.iloc[i]['Attributes']['InstitutionsAddresses']) != len(mycoordslist):
      c += 1
      print(i)
      print(len(df_school.iloc[i]['Attributes']['InstitutionsAddresses']), len(mycoordslist))
    else:
      m += 1
      print(len(df_school.iloc[i]['Attributes']['InstitutionsAddresses']), len(mycoordslist))
  except BaseException:
    pass
print(c, m)


In [52]:
df_school_transform = gpd.GeoDataFrame(columns=['id', 'short_name', 'full_name', 'eoid', 'website', 'orgtype', 'address', 'geometry'])
for i in range(df_school.shape[0]):
  short_name = df_school.iloc[i]['Attributes']['ShortName']
  full_name = df_school.iloc[i]['Attributes']['FullName']
  eoId = df_school.iloc[i]['Attributes']['IDEKIS']
  website = df_school.iloc[i]['Attributes']['WebSite']
  orgtype = df_school.iloc[i]['Attributes']['OrgType']
  if str(type(df_school.iloc[i]['geometry'])).split("'")[1] == 'shapely.geometry.multipolygon.MultiPolygon':
    address_list = []
    mycoordslist = df_school.iloc[i]['geometry'].geoms
    geom_list = []
    for j in range(len(df_school.iloc[i]['Attributes']['InstitutionsAddresses'])):
      address = df_school.iloc[i]['Attributes']['InstitutionsAddresses'][j]['Address']
      geom = mycoordslist[j]
      df_school_transform.loc[len(df_school_transform.index)] = [len(newdf.index), short_name, full_name, eoId, website, orgtype, address, geom]
  elif str(type(df_school.iloc[i]['geometry'])).split("'")[1] == 'shapely.geometry.polygon.Polygon':
    for j in range(len(df_school.iloc[i]['Attributes']['InstitutionsAddresses'])):
      address = df_school.iloc[i]['Attributes']['InstitutionsAddresses'][j]['Address']
    geom = df_school.iloc[i]['geometry']
    df_school_transform.loc[len(df_school_transform.index)] = [len(newdf.index), short_name, full_name, eoId, website, orgtype, address, geom]

In [53]:
df_school_transform

,id,short_name,full_name,eoid,website,orgtype,address,geometry
0,15,ГБОУ ДО ДТДМ «Хорошево»,Государственное бюджетное образовательное учре...,13072,dtim.mskobr.ru,организация дополнительного образования,"Российская Федерация, город Москва, внутригоро...","POLYGON ((37.47940 55.78360, 37.47888 55.78369..."
1,15,ГБУ «Лаборатория путешествий»,Государственное бюджетное учреждение города Мо...,13258,lab-putesh.mskobr.ru,прочее,"Российская Федерация, город Москва, внутригоро...","POLYGON ((37.67580 55.73956, 37.67559 55.73959..."
2,15,ГБПОУ колледж «Царицыно»,Государственное бюджетное профессиональное обр...,11708,collegetsaritsyno.mskobr.ru,профессиональная образовательная организация,"Российская Федерация, город Москва, внутригоро...","POLYGON ((37.71597 55.62112, 37.71596 55.62113..."
3,15,ГБПОУ колледж «Царицыно»,Государственное бюджетное профессиональное обр...,11708,collegetsaritsyno.mskobr.ru,профессиональная образовательная организация,"город Москва, улица Генерала Белова, дом 6","POLYGON ((37.71445 55.62387, 37.71517 55.62400..."
4,15,ГБПОУ колледж «Царицыно»,Государственное бюджетное профессиональное обр...,11708,collegetsaritsyno.mskobr.ru,профессиональная образовательная организация,"Российская Федерация, город Москва, внутригоро...","POLYGON ((37.69638 55.61347, 37.69685 55.61354..."
...,...,...,...,...,...,...,...,...
4954,15,ГБОУ школа № 1512 им.Алии Молдагуловой,Государственное бюджетное общеобразовательное ...,14577,sch1512.mskobr.ru,общеобразовательная организация,"город Москва, Вешняковская улица, дом 25А","POLYGON ((37.80565 55.72440, 37.80621 55.72480..."
4955,15,ГБОУ школа № 1512 им.Алии Молдагуловой,Государственное бюджетное общеобразовательное ...,14577,sch1512.mskobr.ru,общеобразовательная организация,"Российская Федерация, город Москва, внутригоро...","POLYGON ((37.81448 55.71972, 37.81427 55.71978..."
4956,15,ГБОУ школа № 1512 им.Алии Молдагуловой,Государственное бюджетное общеобразовательное ...,14577,sch1512.mskobr.ru,общеобразовательная организация,"Российская Федерация, город Москва, внутригоро...","POLYGON ((37.82124 55.72249, 37.82124 55.72252..."
4957,15,ГБОУ школа № 1512 им.Алии Молдагуловой,Государственное бюджетное общеобразовательное ...,14577,sch1512.mskobr.ru,общеобразовательная организация,"город Москва, улица Молдагуловой, дом 30А","POLYGON ((37.82364 55.71749, 37.82364 55.71752..."


In [54]:
unik_org = set(df_school_transform['orgtype'])
unik_org

{'дошкольная образовательная организация',
 'образовательная организация высшего образования',
 'общеобразовательная организация',
 'организация дополнительного образования',
 'организация дополнительного профессионального образования',
 'профессиональная образовательная организация',
 'прочее'}

In [55]:
save_list = ['дошкольная образовательная организация', 'общеобразовательная организация']

In [56]:
df_school_transform = df_school_transform.loc[df_school_transform['orgtype'].isin(save_list)]

In [57]:
unik_org = set(df_school_transform['orgtype'])
unik_org

{'дошкольная образовательная организация', 'общеобразовательная организация'}

In [58]:
df_school_transform

,id,short_name,full_name,eoid,website,orgtype,address,geometry
214,15,ГБОУ школа № 1366,Государственное бюджетное общеобразовательное ...,13782,sch1366uv.mskobr.ru,общеобразовательная организация,"Российская Федерация, город Москва, внутригоро...","POLYGON ((37.94455 55.71038, 37.94418 55.71047..."
262,15,ГБДОУ детский сад № 754 «Солнышко»,Государственное бюджетное дошкольное образоват...,13296,dou754.mskobr.ru,дошкольная образовательная организация,"Российская Федерация, город Москва, внутригоро...","POLYGON ((37.45077 55.73314, 37.45052 55.73313..."
270,15,ГБДОУ детский сад № 766,Государственное бюджетное дошкольное образоват...,13620,dspo766.mskobr.ru,дошкольная образовательная организация,"Российская Федерация, город Москва, внутригоро...","POLYGON ((37.56174 55.81049, 37.56174 55.81046..."
287,15,ГБОУ школа № 727,Государственное бюджетное общеобразовательное ...,13078,727.mskobr.ru,общеобразовательная организация,"город Москва, Волоколамское шоссе, дом 71, стр...","POLYGON ((37.42976 55.81953, 37.42933 55.81964..."
296,15,ГБОУ школа № 1404 «Гамма»,Государственное бюджетное общеобразовательное ...,13733,gym1404.mskobr.ru,общеобразовательная организация,"Российская Федерация, город Москва, внутригоро...","POLYGON ((37.68578 55.79812, 37.68577 55.79814..."
...,...,...,...,...,...,...,...,...
4954,15,ГБОУ школа № 1512 им.Алии Молдагуловой,Государственное бюджетное общеобразовательное ...,14577,sch1512.mskobr.ru,общеобразовательная организация,"город Москва, Вешняковская улица, дом 25А","POLYGON ((37.80565 55.72440, 37.80621 55.72480..."
4955,15,ГБОУ школа № 1512 им.Алии Молдагуловой,Государственное бюджетное общеобразовательное ...,14577,sch1512.mskobr.ru,общеобразовательная организация,"Российская Федерация, город Москва, внутригоро...","POLYGON ((37.81448 55.71972, 37.81427 55.71978..."
4956,15,ГБОУ школа № 1512 им.Алии Молдагуловой,Государственное бюджетное общеобразовательное ...,14577,sch1512.mskobr.ru,общеобразовательная организация,"Российская Федерация, город Москва, внутригоро...","POLYGON ((37.82124 55.72249, 37.82124 55.72252..."
4957,15,ГБОУ школа № 1512 им.Алии Молдагуловой,Государственное бюджетное общеобразовательное ...,14577,sch1512.mskobr.ru,общеобразовательная организация,"город Москва, улица Молдагуловой, дом 30А","POLYGON ((37.82364 55.71749, 37.82364 55.71752..."


In [62]:
    #Функция для вычисления центроида для объектов
def calculate_centroid(df):
        centroid_list_longitude = []
        centroid_list_latitude = []
        for i in range(df.shape[0]):
            centroid_x = 0
            centroid_y = 0
            counter = 0
            if str(type(df.iloc[i]['geometry'])).split("'")[1] == 'shapely.geometry.polygon.Polygon':
                longitude, latitude = df.iloc[i]['geometry'].exterior.coords.xy
                for j in range(len(longitude)):
                    counter += 1
                    centroid_x += longitude[j]
                    centroid_y += latitude[j]
                centroid_x = centroid_x / counter
                centroid_y = centroid_y / counter

            elif str(type(df.iloc[i]['geometry'])).split("'")[1] == 'shapely.geometry.point.Point':
                centroid_x = list(df.iloc[i]['geometry'].coords)[0][0]
                centroid_y = list(df.iloc[i]['geometry'].coords)[0][1]

            elif str(type(df.iloc[i]['geometry'])).split("'")[1] == 'shapely.geometry.multipolygon.MultiPolygon':
                mycoordslist = [list(x.exterior.coords) for x in df.iloc[i]['geometry'].geoms]
                for j in mycoordslist:
                    for k in range(len(j)):
                        counter += 1
                        centroid_x += j[k][0]
                        centroid_y += j[k][1]
                centroid_x = centroid_x / counter
                centroid_y = centroid_y / counter

            elif str(type(df.iloc[i]['geometry'])).split("'")[1] == 'shapely.geometry.linestring.LineString':
                longitude, latitude = df.iloc[i]['geometry'].coords.xy
                for j in range(len(longitude)):
                    counter += 1
                    centroid_x += longitude[j]
                    centroid_y += latitude[j]
                centroid_x = centroid_x / counter
                centroid_y = centroid_y / counter

            centroid_list_longitude.append(centroid_x)
            centroid_list_latitude.append(centroid_y)

        return centroid_list_latitude, centroid_list_longitude

In [60]:
def calculate_area(border_value, median_value, df):
        area_list = []
        geod = Geod(ellps="WGS84")
        for i in range(df.shape[0]):
            if str(type(df.iloc[i]['geometry'])).split("'")[1] == 'shapely.geometry.point.Point':
                area = median_value

            else:
                poly = wkt.loads(str(df.iloc[i]['geometry']))
                area = abs(geod.geometry_area_perimeter(poly)[0])

            if area > border_value:
                area = median_value
            area_list.append(area)
        return area_list

In [63]:
df_school_transform['centroid latitude'], df_school_transform['centroid longitude'] = calculate_centroid(df_school_transform)


/usr/local/lib/python3.9/dist-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/usr/local/lib/python3.9/dist-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [64]:
df_school_transform

,id,short_name,full_name,eoid,website,orgtype,address,geometry,centroid latitude,centroid longitude
214,15,ГБОУ школа № 1366,Государственное бюджетное общеобразовательное ...,13782,sch1366uv.mskobr.ru,общеобразовательная организация,"Российская Федерация, город Москва, внутригоро...","POLYGON ((37.94455 55.71038, 37.94418 55.71047...",55.710716,37.944504
262,15,ГБДОУ детский сад № 754 «Солнышко»,Государственное бюджетное дошкольное образоват...,13296,dou754.mskobr.ru,дошкольная образовательная организация,"Российская Федерация, город Москва, внутригоро...","POLYGON ((37.45077 55.73314, 37.45052 55.73313...",55.733190,37.451302
270,15,ГБДОУ детский сад № 766,Государственное бюджетное дошкольное образоват...,13620,dspo766.mskobr.ru,дошкольная образовательная организация,"Российская Федерация, город Москва, внутригоро...","POLYGON ((37.56174 55.81049, 37.56174 55.81046...",55.810621,37.561923
287,15,ГБОУ школа № 727,Государственное бюджетное общеобразовательное ...,13078,727.mskobr.ru,общеобразовательная организация,"город Москва, Волоколамское шоссе, дом 71, стр...","POLYGON ((37.42976 55.81953, 37.42933 55.81964...",55.819829,37.429659
296,15,ГБОУ школа № 1404 «Гамма»,Государственное бюджетное общеобразовательное ...,13733,gym1404.mskobr.ru,общеобразовательная организация,"Российская Федерация, город Москва, внутригоро...","POLYGON ((37.68578 55.79812, 37.68577 55.79814...",55.798140,37.685869
...,...,...,...,...,...,...,...,...,...,...
4954,15,ГБОУ школа № 1512 им.Алии Молдагуловой,Государственное бюджетное общеобразовательное ...,14577,sch1512.mskobr.ru,общеобразовательная организация,"город Москва, Вешняковская улица, дом 25А","POLYGON ((37.80565 55.72440, 37.80621 55.72480...",55.724468,37.806183
4955,15,ГБОУ школа № 1512 им.Алии Молдагуловой,Государственное бюджетное общеобразовательное ...,14577,sch1512.mskobr.ru,общеобразовательная организация,"Российская Федерация, город Москва, внутригоро...","POLYGON ((37.81448 55.71972, 37.81427 55.71978...",55.720101,37.814408
4956,15,ГБОУ школа № 1512 им.Алии Молдагуловой,Государственное бюджетное общеобразовательное ...,14577,sch1512.mskobr.ru,общеобразовательная организация,"Российская Федерация, город Москва, внутригоро...","POLYGON ((37.82124 55.72249, 37.82124 55.72252...",55.722242,37.821301
4957,15,ГБОУ школа № 1512 им.Алии Молдагуловой,Государственное бюджетное общеобразовательное ...,14577,sch1512.mskobr.ru,общеобразовательная организация,"город Москва, улица Молдагуловой, дом 30А","POLYGON ((37.82364 55.71749, 37.82364 55.71752...",55.717618,37.823988


In [73]:
border_value = 10000 #Все площади что больше этой заменяются на median_value, подобрано имперически
        # необходимо в двух случаях: когда полигон является точкой и не получается узнать площадь
        # и когда школа выкачивается вместе с окружающей ее территорией, что делает ее площадь во много раз больше.
        # Значение является медианой для площадей школ, площади которых указаны верно
median_value = 1307
df_school_transform['area'] = calculate_area(border_value, median_value, df_school_transform)

/usr/local/lib/python3.9/dist-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [71]:
df_school_transform

,id,short_name,full_name,eoid,website,orgtype,address,geometry,centroid latitude,centroid longitude,area
214,15,ГБОУ школа № 1366,Государственное бюджетное общеобразовательное ...,13782,sch1366uv.mskobr.ru,общеобразовательная организация,"Российская Федерация, город Москва, внутригоро...","POLYGON ((37.94455 55.71038, 37.94418 55.71047...",55.710716,37.944504,1983.515701
262,15,ГБДОУ детский сад № 754 «Солнышко»,Государственное бюджетное дошкольное образоват...,13296,dou754.mskobr.ru,дошкольная образовательная организация,"Российская Федерация, город Москва, внутригоро...","POLYGON ((37.45077 55.73314, 37.45052 55.73313...",55.733190,37.451302,2814.411855
270,15,ГБДОУ детский сад № 766,Государственное бюджетное дошкольное образоват...,13620,dspo766.mskobr.ru,дошкольная образовательная организация,"Российская Федерация, город Москва, внутригоро...","POLYGON ((37.56174 55.81049, 37.56174 55.81046...",55.810621,37.561923,1536.868115
287,15,ГБОУ школа № 727,Государственное бюджетное общеобразовательное ...,13078,727.mskobr.ru,общеобразовательная организация,"город Москва, Волоколамское шоссе, дом 71, стр...","POLYGON ((37.42976 55.81953, 37.42933 55.81964...",55.819829,37.429659,1614.008824
296,15,ГБОУ школа № 1404 «Гамма»,Государственное бюджетное общеобразовательное ...,13733,gym1404.mskobr.ru,общеобразовательная организация,"Российская Федерация, город Москва, внутригоро...","POLYGON ((37.68578 55.79812, 37.68577 55.79814...",55.798140,37.685869,124.629469
...,...,...,...,...,...,...,...,...,...,...,...
4954,15,ГБОУ школа № 1512 им.Алии Молдагуловой,Государственное бюджетное общеобразовательное ...,14577,sch1512.mskobr.ru,общеобразовательная организация,"город Москва, Вешняковская улица, дом 25А","POLYGON ((37.80565 55.72440, 37.80621 55.72480...",55.724468,37.806183,2189.268262
4955,15,ГБОУ школа № 1512 им.Алии Молдагуловой,Государственное бюджетное общеобразовательное ...,14577,sch1512.mskobr.ru,общеобразовательная организация,"Российская Федерация, город Москва, внутригоро...","POLYGON ((37.81448 55.71972, 37.81427 55.71978...",55.720101,37.814408,2220.198321
4956,15,ГБОУ школа № 1512 им.Алии Молдагуловой,Государственное бюджетное общеобразовательное ...,14577,sch1512.mskobr.ru,общеобразовательная организация,"Российская Федерация, город Москва, внутригоро...","POLYGON ((37.82124 55.72249, 37.82124 55.72252...",55.722242,37.821301,1686.246401
4957,15,ГБОУ школа № 1512 им.Алии Молдагуловой,Государственное бюджетное общеобразовательное ...,14577,sch1512.mskobr.ru,общеобразовательная организация,"город Москва, улица Молдагуловой, дом 30А","POLYGON ((37.82364 55.71749, 37.82364 55.71752...",55.717618,37.823988,912.944839


In [74]:
list(df_school_transform['area']).count(median_value)

14

In [77]:
df_school_transform.loc[df_school_transform['area'] == median_value]

,id,short_name,full_name,eoid,website,orgtype,address,geometry,centroid latitude,centroid longitude,area
570,15,ГАОУ школа № 548,Государственное автономное общеобразовательное...,13866,cou548.mskobr.ru,общеобразовательная организация,"Российская Федерация, город Москва, внутригоро...","POLYGON ((37.63949 55.70035, 37.63910 55.70039...",55.701020,37.638499,1307.0
596,15,ГБОУ школа № 1621,Государственное бюджетное общеобразовательное ...,13867,sch1621.mskobr.ru,общеобразовательная организация,"Российская Федерация, город Москва, внутригоро...","POLYGON ((37.53951 55.69229, 37.54029 55.69184...",55.692333,37.541193,1307.0
960,15,ГБОУ школа № 1409,Государственное бюджетное общеобразовательное ...,13801,gym1409s-new.mskobr.ru,общеобразовательная организация,"Российская Федерация, город Москва, внутригоро...","POLYGON ((37.53879 55.78674, 37.53864 55.78656...",55.786557,37.539793,1307.0
1476,15,ГБОУ школа № 2086,Государственное бюджетное общеобразовательное ...,13282,sch2086uz.mskobr.ru,общеобразовательная организация,"Российская Федерация, город Москва, внутригоро...","POLYGON ((37.55179 55.69493, 37.55156 55.69506...",55.695871,37.551456,1307.0
1480,15,ГБОУ школа № 2086,Государственное бюджетное общеобразовательное ...,13282,sch2086uz.mskobr.ru,общеобразовательная организация,"Российская Федерация, город Москва, внутригоро...","POLYGON ((37.54507 55.69044, 37.54508 55.69055...",55.689871,37.545370,1307.0
1520,15,ГБОУ школа № 508,Государственное бюджетное общеобразовательное ...,12933,sch508u.mskobr.ru,общеобразовательная организация,"Российская Федерация, город Москва, внутригоро...","POLYGON ((37.66254 55.60468, 37.66341 55.60421...",55.604653,37.660929,1307.0
1625,15,ГБОУ школа № 1533 «ЛИТ»,Государственное бюджетное общеобразовательное ...,13720,lyc1533.mskobr.ru,общеобразовательная организация,"Российская Федерация, город Москва, внутригоро...","POLYGON ((37.53951 55.69229, 37.54029 55.69184...",55.692322,37.541148,1307.0
1680,15,ГБОУ школа № 2070,Государственное бюджетное общеобразовательное ...,14484,sch2070.mskobr.ru,общеобразовательная организация,"Российская Федерация, город Москва, внутригоро...","POLYGON ((37.49463 55.56809, 37.49493 55.56752...",55.567622,37.493640,1307.0
2059,15,ГБОУ школа № 2048,Государственное бюджетное общеобразовательное ...,12960,2048.mskobr.ru,общеобразовательная организация,"город Москва, Покровская улица, дом 16А","POLYGON ((37.93876 55.70369, 37.93822 55.70321...",55.703599,37.938057,1307.0
3539,15,ГБОУ школа № 1708,Государственное бюджетное общеобразовательное ...,12727,sch1708uz.mskobr.ru,общеобразовательная организация,"Российская Федерация, город Москва, внутригоро...","POLYGON ((37.53783 55.69039, 37.53795 55.69032...",55.691080,37.537297,1307.0


In [78]:
df_school_transform.to_file('school_datamos_transform', driver="GeoJSON")